This notebook explores the use of the bootstrap to create confidence intervals for any statistic of interest that is estimated from data.

In [ ]:
import sys
from collections import Counter
from sklearn import preprocessing
from sklearn import linear_model
import pandas as pd
from scipy import sparse
import numpy as np
from math import sqrt 
from scipy.stats import norm
from random import choices

In [ ]:
def read_data(filename):
    X=[]
    Y=[]
    with open(filename, encoding="utf-8") as file:
        for line in file:
            cols=line.rstrip().split("\t")
            label=cols[0]
            text=cols[1]
            # assumes text is tokenized
            X.append(text)
            Y.append(label)
    return X, Y

In [ ]:
# Change this to the directory with your data (from the CheckData_TODO.ipynb exercise).  
# The directory should contain train.tsv, dev.tsv and test.tsv
directory="../data/text_classification_sample_data"

In [ ]:
trainX, trainY=read_data("%s/train.tsv" % directory)
devX, devY=read_data("%s/dev.tsv" % directory)

In [ ]:
# Here's a sample dictionary we can create by inspecting the output of the Mann-Whitney test (in 2.compare/)
dem_dictionary=set(["republican","cut", "opposition"])
repub_dictionary=set(["growth","economy"])

def political_dictionary_feature(tokens):
    feats={}
    for word in tokens:
        if word in dem_dictionary:
            feats["word_in_dem_dictionary"]=1
        if word in repub_dictionary:
            feats["word_in_repub_dictionary"]=1
    return feats

In [ ]:
def build_features(trainX, feature_functions):
    data=[]
    for doc in trainX:
        feats={}

        # sample text data is already tokenized; if yours is not, do so here
        tokens=doc.split(" ")
        
        for function in feature_functions:
            feats.update(function(tokens))

        data.append(feats)
    return data

In [ ]:
# This helper function converts a dictionary of feature names to unique numerical ids
def create_vocab(data):
    feature_vocab={}
    idx=0
    for doc in data:
        for feat in doc:
            if feat not in feature_vocab:
                feature_vocab[feat]=idx
                idx+=1
                
    return feature_vocab

In [ ]:
# This helper function converts a dictionary of feature names to a sparse representation
# that we can fit in a scikit-learn model.  This is important because almost all feature 
# values will be 0 for most documents (note: why?), and we don't want to save them all in 
# memory.

def features_to_ids(data, feature_vocab):
    new_data=sparse.lil_matrix((len(data), len(feature_vocab)))
    for idx,doc in enumerate(data):
        for f in doc:
            if f in feature_vocab:
                new_data[idx,feature_vocab[f]]=doc[f]
    return new_data

In [ ]:
# This function trains a model and returns the predicted and true labels for test data
def evaluate(trainX, devX, trainY, devY, feature_functions):
    trainX_feat=build_features(trainX, feature_functions)
    devX_feat=build_features(devX, feature_functions)

    # just create vocabulary from features in *training* data
    feature_vocab=create_vocab(trainX_feat)

    trainX_ids=features_to_ids(trainX_feat, feature_vocab)
    devX_ids=features_to_ids(devX_feat, feature_vocab)
    
    le=preprocessing.LabelEncoder()
    le.fit(trainY)

    trainY=le.transform(trainY)
    devY=le.transform(devY)
    
    print ("Class 1 is %s" % le.inverse_transform([1]))
    
    logreg = linear_model.LogisticRegression(C=1.0, solver='lbfgs', penalty='l2', max_iter=10000)
    logreg.fit(trainX_ids, trainY)
    print ("Accuracy: %.3f"  % logreg.score(devX_ids, devY))
    predictions=logreg.predict(devX_ids)
    
    return (predictions, devY)

In [ ]:
def binomial_confidence_intervals(predictions, truth, confidence_level=0.95):
    correct=[]
    for pred, gold in zip(predictions, truth):
        correct.append(int(pred==gold))
        
    success_rate=np.mean(correct)

    # two-tailed test
    critical_value=(1-confidence_level)/2
    # ppf finds z such that p(X < z) = critical_value
    z_alpha=-1*norm.ppf(critical_value)
    
    # the standard error is the square root of the variance/sample size
    # the variance for a binomial test is p*(1-p)
    standard_error=sqrt((success_rate*(1-success_rate))/len(correct))

    lower=success_rate-z_alpha*standard_error
    upper=success_rate+z_alpha*standard_error
    print("%.3f, %s%% Confidence interval: [%.3f,%.3f]" % (success_rate, confidence_level*100, lower, upper))

In [ ]:
def accuracy(truth, predictions):
    correct=0.
    for idx in range(len(truth)):
        g=truth[idx]
        p=predictions[idx]
        if g == p:
            correct+=1
    return correct/len(truth)

Specify features for model and train logistic regression

In [ ]:
features=[political_dictionary_feature]
predictions, truth=evaluate(trainX, devX, trainY, devY, features)

In [ ]:
binomial_confidence_intervals(predictions, truth, confidence_level=0.95)

Q1: Implement the bootstrap to create confidence intervals at a specified confidence level for any function `metric(truth, predictions)` where *truth* is an array of true labels for a set of data points, and *predictions* is an array of predicted labels for those same points.  See `accuracy(truth, predictions)` above for an example of a metric that should be supported.  `bootstrap` should return a tuple of (lower, median, upper), where *lower* is the lower confidence bound, *upper* is the upper confidence bound, and *median* is the median value of the metric among the bootstrap resamples.  Hint: see [np.percentile](https://docs.scipy.org/doc/numpy/reference/generated/numpy.percentile.html).

In [ ]:
def bootstrap(gold, predictions, metric, B=10000, confidence_level=0.95):
    critical_value=(1-confidence_level)/2
    lower_sig=100*critical_value
    upper_sig=100*(1-critical_value)
    data=[]
    for g, p in zip(gold, predictions):
        data.append([g,p])

    accuracies=[]
    
    for b in range(B):
        choice=choices(data, k=len(data))
        choice=np.array(choice)
        accuracy=metric(choice[:,0], choice[:,1])
        
        accuracies.append(accuracy)
    
    percentiles=np.percentile(accuracies, [lower_sig, 50, upper_sig])
    
    lower=percentiles[0]
    median=percentiles[1]
    upper=percentiles[2]
    
    return lower, median, upper


Q2: Use your bootstrap implementation to generate confidence intervals for accuracy.  How do these compare to the parametric intervals above?

In [ ]:
confidence_level=0.95
lower, median,upper=bootstrap(truth, predictions, accuracy, B=10000,confidence_level=confidence_level)
print("%.3f, %s%% Bootstrap confidence interval: [%.3f, %.3f]" % (median, confidence_level*100, lower, upper))

Q3: Implement the F1 score for binary data.  Calculate F1 as the harmonic mean of precision and recall for the positive class (i.e., y=1).

In [ ]:
def F1(truth, predictions):
    correct=0.
    trials=0.
    trues=0.
    for idx in range(len(truth)):
        g=truth[idx]
        p=predictions[idx]
        if g == p and g == 1:
            correct+=1
        if g == 1:
            trues+=1
        if p == 1:
            trials+=1
            
    precision=correct/trials if trials > 0 else 0
    recall=correct/trues if trues > 0 else 0
    f=(2*precision*recall)/(precision+recall) if (precision+recall) > 0 else 0
    return f

Q4: Use your bootstrap implementation to generate confidence intervals for the F1 score. 

In [ ]:
confidence_level=0.95
lower, median,upper=bootstrap(truth, predictions, F1, B=10000,confidence_level=confidence_level)
print("%.3f, %s%% Bootstrap confidence interval: [%.3f, %.3f]" % (median, confidence_level*100, lower, upper))